In [1]:
import contextlib
import os
from time import sleep

import qibo
from qibo import gates
from qibo.models.circuit import Circuit

from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

## Connect to the Qibo Service API

In [2]:
os.environ["QIBOCONNECTION_ENVIRONMENT"] = "development"
configuration = ConnectionConfiguration(username="qili-admin-test", api_key="45843bfc-c404-42ad-abc5-070dk7869420")
qibo_api = API(configuration=configuration)

[qibo-connection] 0.9.0|INFO|2023-05-31 12:09:38]: Storing personal qibo configuration...


## Load an already existing connection to the Qilimanjaro Service API

In [3]:
qibo_api = API()

## Check a connection is alive

In [4]:
qibo_api.ping()

'OK'

## List all registered devices

In [34]:
devices = qibo_api.list_devices()
devices

<Devices[3]:
{
  "device_id": 1,
  "device_name": "dev_sim",
  "status": "online",
  "availability": "available",
  "characteristics": {
    "type": "simulator",
    "cpu": null,
    "gpu": null,
    "os": null,
    "kernel": null,
    "ram": null
  }
}
{
  "device_id": 9,
  "device_name": "dev_quan",
  "status": "online",
  "availability": "available",
  "characteristics": {
    "type": "quantum"
  },
  "calibration_details": {
    "t1": 10,
    "frequency": 988
  }
}
{
  "device_id": 10,
  "device_name": "offline",
  "status": "offline",
  "availability": "available"
}

## Selecting a specific device

In [33]:
qibo_api.select_device_id(device_id=9)

[qibo-connection] 0.10.0|INFO|2023-05-31 11:28:49]: Device dev_quan selected.


## Creating an experiment to be executed on a remote device

In [28]:
qibo.set_backend("numpy")

[Qibo 0.1.12.dev0|INFO|2023-05-31 11:28:25]: Using numpy backend on /CPU:0


In [29]:
circuit = Circuit(1)
circuit.add(gates.H(0))
circuit.add(gates.M(0))

MeasurementResult(qubits=(0,), nshots=0)

## Remote execution and getting a job identifier

In [35]:
job_id = qibo_api.execute(circuit=circuit)

In [36]:
job_id

[5420]

## User retrieves the Qibo result, but it is still pending

In [37]:
result = qibo_api.get_result(job_id=job_id[0])

[qibo-connection] 0.10.0|ERROR|2023-05-31 11:29:04]: Your job with id 5420 failed.


## When job is executed remotely, a user can retrieve the actual result

In [38]:
sleep(1)
result = qibo_api.get_result(job_id=job_id)

TypeError: type of argument "job_id" must be int; got list instead

In [15]:
result.state()

AttributeError: 'NoneType' object has no attribute 'state'

In [ ]:
result.frequencies(binary=True, registers=True)

In [ ]:
circuit.draw()

In [ ]:
qibo_api.release_device(device_id=9)

In [ ]:
devices._devices[0].id

## Block and release a remote device to operate on it

In [ ]:
qibo_api.block_device_id(device_id=1)

# Do stuff with the device, knowing no one else will access it

qibo_api.release_device(device_id=1)

## Trying to block a blocked device will raise an exception

In [ ]:
with contextlib.suppress(Exception):
    qibo_api.block_device_id(device_id=1)
    qibo_api.block_device_id(device_id=1)

qibo_api.release_device(device_id=1)

In [5]:
response = qibo_api._get_list_jobs_response()

TypeError: ListingJobResponse.__init__() got an unexpected keyword argument 'result'

In [7]:
response

[SavedExperimentListingItemResponse(name='SaveTest', experiment='eyJwbGF0Zm9ybSI6IHsic2V0dGluZ3MiOiB7ImlkXyI6IDAsICJjYXRlZ29yeSI6ICJwbGF0Zm9ybSIsICJhbGlhcyI6IG51bGwsICJuYW1lIjogImdhbGFkcmllbCIsICJkZWxheV9iZXR3ZWVuX3B1bHNlcyI6IDAsICJkZWxheV9iZWZvcmVfcmVhZG91dCI6IDgwLjAsICJtYXN0ZXJfYW1wbGl0dWRlX2dhdGUiOiAxLCAibWFzdGVyX2R1cmF0aW9uX2dhdGUiOiAxMDAsICJnYXRlcyI6IFt7Im5hbWUiOiAiTSIsICJhbXBsaXR1ZGUiOiAibWFzdGVyX2FtcGxpdHVkZV9nYXRlIiwgInBoYXNlIjogMCwgImR1cmF0aW9uIjogMjAwMCwgInNoYXBlIjogeyJuYW1lIjogInJlY3Rhbmd1bGFyIn19LCB7Im5hbWUiOiAiSSIsICJhbXBsaXR1ZGUiOiAwLCAicGhhc2UiOiAwLCAiZHVyYXRpb24iOiAwLCAic2hhcGUiOiB7Im5hbWUiOiAicmVjdGFuZ3VsYXIifX0sIHsibmFtZSI6ICJYIiwgImFtcGxpdHVkZSI6ICJtYXN0ZXJfYW1wbGl0dWRlX2dhdGUiLCAicGhhc2UiOiAwLCAiZHVyYXRpb24iOiAibWFzdGVyX2R1cmF0aW9uX2dhdGUiLCAic2hhcGUiOiB7Im5hbWUiOiAiZHJhZyIsICJudW1fc2lnbWFzIjogNCwgImRyYWdfY29lZmZpY2llbnQiOiAwfX0sIHsibmFtZSI6ICJZIiwgImFtcGxpdHVkZSI6ICJtYXN0ZXJfYW1wbGl0dWRlX2dhdGUiLCAicGhhc2UiOiAxLjU3MDc5NjMyNjc5NDg5NjYsICJkdXJhdGlvbiI6ICJtYXN0ZXJfZHV

response